## Step 1: Setup and Library Imports

In [1]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq  # Assuming you have access to Groq's client
import os

def load_json_data(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def create_tfidf_vectors(data):
    documents = [entry['question'] + " " + entry['code'] for entry in data]  # Context includes both code and the question
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)
    return vectorizer, tfidf_matrix


## Step 2: Implementing Document Retrieval

In [2]:
def retrieve_documents(query, data, vectorizer, tfidf_matrix):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_n_indices = np.argsort(-similarities)[:5]  # Retrieves the top 5 documents
    return [data[i] for i in top_n_indices if similarities[i] > 0]


## Step 3: Generating Responses with Chain of Thought

In [3]:
def generate_response_with_chain_of_thought(query, documents):
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    # Construct a reasoning path from the documents
    reasoning = "Based on the provided code and context, "
    context = " ".join([f"{doc['question']} Hence, {doc['answer']}." for doc in documents])
    reasoning += context
    combined_input = f"Question: {query} {reasoning} Therefore, "
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": combined_input,
            }
        ],
        model="llama-3.3-70b-versatile",
    )
    return response.choices[0].message.content
